In [10]:
import pandas as pd
import numpy as np
import re
import json
import torch
import random
from probes import *
import numpy as np
import pickle
import matplotlib.pyplot as plt
torch.set_printoptions(profile="full")
import re
import pandas as pd
pd.set_option('display.max_columns', None)

In [11]:
import os
os.getcwd()

'/Users/jessicakahn/Documents/repos/probing_classifiers/goodreads_data'

In [12]:
import ast

In [13]:
df = pd.read_csv("goodreads_samples.csv")
user_id_map = pd.read_csv("user_id_map.csv")
book_id_map = pd.read_csv("book_id_map.csv")
with open('author_data.json', 'r') as f:
    author_dict = json.load(f)
with open('book_data.json', 'r') as f:
    book_dict = json.load(f)
merged_df = pd.merge(df, book_id_map, left_on ='book_id',right_on='book_id_csv', how='left')
totals = merged_df.groupby('user_id')['rating'].agg(lambda x: (x != 0).sum()).reset_index()

totals.columns = ['user_id','rating_count']
merged_df2 = pd.merge(merged_df, totals[totals.rating_count >= 3], on ='user_id', how='inner')
result_df = merged_df2.groupby('user_id')['book_id_y'].agg(list).reset_index()

In [14]:
# with open('steered_output_100.json', 'r') as f:
#     output = json.load(f)
# result_df.head(2)

In [8]:
# result_df.head()
outputs = torch.load('output_dict_500.pt', map_location=torch.device('cpu'))

In [20]:
outputs[82124]['baseline_text']

'system\n\nCutting Knowledge Date: December 2023\nToday Date: 13 Nov 2025\n\nuser\n\nHi, I\'ve read and enjoyed the following books: Nerd in Shining Armor (Nerds, #1) by Vicki Lewis Thompson,  \n        The Snow Queen (Five Hundred Kingdoms, #4) by Mercedes Lackey,Gossip by Beth Gutcheon,Sequins, Secrets, and Silver Linings (Threads, #1) by Sophia Bennett  and Too Many Curses by A. Lee Martinez.  Please recommend new books based on the user\'s reading preferences and only return the 5 books you recommend in JSON format like {"Books": {\'title\':..., \'author\':...}}, and nothing else.assistant\n\n{\n  "Books": [\n    {"title": "The Princess and the Goblin", "author": "George MacDonald"},\n    {"title": "The Siren", "author": "Kiera Cass"},\n    {"title": "Ella Enchanted", "author": "Gail Carson Levine"},\n    {"title": "The Wicked City", "author": "Gail Carriger"},\n    {"title": "The Mediator: Girl in Peril", "author": "Meg Cab'

In [15]:
# outputs.keys()
user_title_dict = {}
for i, row in result_df.iterrows():
    user_title_dict[row['user_id']] = [book_dict[str(i)]['title_without_series'] for i in row['book_id_y']]

In [21]:
str1 = "system\n\nCutting Knowledge Date: December 2023\nToday Date: 08 Nov 2025\n\nuser\n\nHi, I\'ve read and enjoyed the following books:"
str1b = "Hi, I\'ve read and enjoyed the following books:"
str2 = """  Only return the 5 books you recommend in JSON format like {"Books": {\'title\':..., \'author\':...}}, and nothing else.assistant\n\n"""
str3 = """Please recommend new books based on the user\'s reading preferences and only return the 5 books you recommend in JSON format like {"Books": {\'title\':..., \'author\':...}}, and nothing else.assistant"""

In [46]:
for k, values in outputs.items():
    values['baseline_recs'] = extract_books(values['baseline_text'].split(str3)[1].replace("\n",""))
    values['steered_recs'] = extract_books(values['steered_text'].split(str3)[1].replace("\n",""))
    values['hist'] = extract_books(values['baseline_text'].split(str3)[0].split(str1b)[1].replace(str1,""))

In [47]:
outputs[66235]['hist']

[{'title': 'Surrender to Me (I Surrender, #2)', 'author': 'Monica  James'},
 {'title': 'Forever My Girl (The Beaumont Series, #1)',
  'author': 'Heidi McLaughlin'},
 {'title': 'Curtis (Coyote Ridge, #1)', 'author': 'Nicole Edwards'},
 {'title': 'TouchStone for play (Story of Us Trilogy, #1)',
  'author': 'Sydney Jamesson  and Rescuing Their Virgin Mate (Pack Wars'}]

In [24]:
for user, values in outputs.items():
    values['titles'] = user_title_dict[user]
    values['baseline_count'] = 0
    values['steered_count'] = 0
    for title in values['titles']:
        if ((title in values['baseline_recs']) and (title not in values['hist'])):
            values['baseline_count'] += 1
        if ((title in values['steered_recs']) and (title not in values['hist'])):
            values['steered_count'] += 1

In [27]:
outputs[66235]['baseline_recs']

'{  "Books": [    {"title": "Him", "author": "Sarah J. Maas"},    {"title": "From Lukov with Love", "author": "Marie Lu"},    {"title": "The Simple Wild", "author": "K.A. Tucker"},    {"title": "Well Met", "author": " Jen DeLuca"},    {"title": "A Royal Pain", "author": "Alicia Street"}  ]}'

In [28]:
outputs[66235]['steered_recs']

'```{  "Books": [    {"title": "Take the Lead", "author": "Vi Keeland"},    {"title": "Wait for You", "author": "J. Lynn"},    {"title": "His Hideaway", "author": "Penny Reid"},    {"title": "The Wall of Winnipeg and Me", "author": "Mariana Zapata"},    {"title": "From Lukov with Love", "author": "Mariana Zapata"}'

In [29]:
outputs[66235]['hist']

' Surrender to Me (I Surrender, #2) by Monica  James,  \n        Forever My Girl (The Beaumont Series, #1) by Heidi McLaughlin,Curtis (Coyote Ridge, #1) by Nicole Edwards,TouchStone for play (Story of Us Trilogy, #1) by Sydney Jamesson  and Rescuing Their Virgin Mate (Pack Wars, #3) by Vella Day.  '

In [30]:
user_title_dict[66235]

['TouchStone for play (Story of Us Trilogy, #1)',
 'Destined Havoc (Havoc, #1)',
 'Curtis (Coyote Ridge, #1)',
 'Surrender to Me (I Surrender, #2)',
 'The Coincidence of Callie & Kayden (The Coincidence, #1)',
 'Enticed By You (by You, #1)',
 'Moto',
 'Rescuing Their Virgin Mate (Pack Wars, #3)',
 'Under the Lights (The Field Party, #2)',
 'Fade into You (Fade, #1)',
 'Dirty Past (The Burke Brothers, #2)',
 'Know Me (Defiant Motorcycle Club, #1)',
 "What Doesn't Destroy Us (The Devil's Dust, #1)",
 'Forever My Girl (The Beaumont Series, #1)']

### Evaluate author gender

In [31]:
author_gender = pd.read_csv('final_dataset.csv')

In [33]:
author_gender.head()

,authorid,name,workcount,fan_count,gender,image_url,about,born,died,influence,average_rate,rating_count,review_count,website,twitter,genre,original_hometown,country,latitude,longitude
0,8409092,Jason Wallace,2,13,male,https://images.gr-assets.com/authors/148926684...,Jason Wallace is related to Tolkien and a desc...,NaN,NaN,NaN,3.74,1028,175,NaN,NaN,NaN,Cheltenham,United Kingdom,51.90006,-2.07972
1,5796406,Rosan Hollak,4,0,unknown,https://s.gr-assets.com/assets/nophoto/user/u_...,NaN,NaN,NaN,NaN,3.73,15,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8421525,Nanna Foss,6,156,female,https://images.gr-assets.com/authors/140908587...,Danish YA writer. <br /><br />I make up storie...,NaN,NaN,NaN,4.35,1172,205,http://www.nannafoss.dk,NaN,"fantasy,fiction,paranormal",NaN,NaN,NaN,NaN
3,158146,Terri Savelle Foy,23,125,female,https://images.gr-assets.com/authors/147569460...,"For years, Terri Savelle Foy’s life was averag...",NaN,NaN,NaN,4.56,1054,151,http://www.terri.com,terrisavellefoy,"religion and spirituality,self help,spirituality",NaN,NaN,NaN,NaN
4,15340731,Vishwas Nangare Patil,1,127,unknown,https://s.gr-assets.com/assets/nophoto/user/u_...,NaN,NaN,NaN,NaN,4.15,725,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
for key, value in outputs.items(): # user IDs
    if (value['baseline_count'] + value['steered_count']) > 0:
        print(key,value['baseline_count'],value['steered_count'])

325723 1 0
141311 0 1


In [48]:
extract_books(outputs[141311]['steered_text'])

[{'title': "Cutting Knowledge Date: December 2023\nToday Date: 13 Nov 2025\n\nuser\n\nHi, I've read and enjoyed the following books: Some Are Born to Sweet Delight",
  'author': 'Nadine Gordimer'},
 {'title': 'Ghost Wars: The Secret History of the CIA, Afghanistan, and bin Laden from the Soviet Invasion to September 10, 2001',
  'author': 'Steve Coll'},
 {'title': 'Blinding Love 2 (Blinding Love, #2)', 'author': 'Liv Bennett'},
 {'title': 'Leaving the Comfort Cafe',
  'author': 'Dawn DeAnna Wilson  and Witch (Wicked'}]

In [51]:
outputs[141311]['baseline_text']

'system\n\nCutting Knowledge Date: December 2023\nToday Date: 13 Nov 2025\n\nuser\n\nHi, I\'ve read and enjoyed the following books: Some Are Born to Sweet Delight by Nadine Gordimer,  \n        Ghost Wars: The Secret History of the CIA, Afghanistan, and bin Laden from the Soviet Invasion to September 10, 2001 by Steve Coll,Blinding Love 2 (Blinding Love, #2) by Liv Bennett,Leaving the Comfort Cafe by Dawn DeAnna Wilson  and Witch (Wicked, #1) by Nancy Holder.  Please recommend new books based on the user\'s reading preferences and only return the 5 books you recommend in JSON format like {"Books": {\'title\':..., \'author\':...}}, and nothing else.assistant\n\n{\n  "Books": [\n    {"title": "Disobedience", "author": "Naomi Alderman"},\n    {"title": "The Looming Tower", "author": "Lawrence Wright"},\n    {"title": "The Seven Husbands of Evelyn Hugo", "author": "Taylor Jenkins Reid"},\n    {"title": "The Other Mrs. Miller", "author": " Allison Dickson"},\n    {"title": "The Invisible L

In [52]:
outputs[141311]['hist']

[{'title': 'Some Are Born to Sweet Delight', 'author': 'Nadine Gordimer'},
 {'title': 'Ghost Wars: The Secret History of the CIA, Afghanistan, and bin Laden from the Soviet Invasion to September 10, 2001',
  'author': 'Steve Coll'},
 {'title': 'Blinding Love 2 (Blinding Love, #2)', 'author': 'Liv Bennett'},
 {'title': 'Leaving the Comfort Cafe',
  'author': 'Dawn DeAnna Wilson  and Witch (Wicked'}]

In [37]:
user_title_dict[141311]

['Men of the Otherworld (Otherworld Stories, #I)',
 'The Fellowship of the Ring (The Lord of the Rings, #1)',
 'Sisters Red (Fairytale Retellings, #1)',
 'Cruel Beauty (Cruel Beauty Universe, #1)',
 'Longbourn',
 'Wondrous Strange (Wondrous Strange, #1)',
 'Crossroads of Twilight (Wheel of Time, #10)',
 'The Rules (Project Paper Doll, #1)',
 'Perdition (Dred Chronicles, #1)',
 'Mirror Sight (Green Rider, #5)',
 'Cress (The Lunar Chronicles, #3)',
 'Feed (Newsflesh Trilogy, #1)',
 'Frost',
 'She-Wolves: The Women Who Ruled England Before Elizabeth']

In [11]:
# (data_dict)[14]
# really need the user ids in the output - next run needs that added

In [12]:
str1 = "system\n\nCutting Knowledge Date: December 2023\nToday Date: 08 Nov 2025\n\nuser\n\nHi, I\'ve read and enjoyed the following books:"
str2 = """  Only return the 3 books you recommend in JSON format like {"Books": {\'title\':..., \'author\':...}}, and nothing else.assistant\n\n"""
str3 = """Only return the 3 books you recommend in JSON format like {"Books": {\'title\':..., \'author\':...}}, and nothing else.assistant"""
pattern = r"\s*(?:,|and)\s*"

In [15]:
# # Extract the recommendations and originals
# for alpha, dic in outputs.items():
#     for user, values in dic.items():
#         values['baseline_recs'] = universal_repeated_key_parser(values['baseline_text'].split(str3)[1].replace("\n",""))
#         values['steered_recs'] = universal_repeated_key_parser(values['steered_text'].split(str3)[1].replace("\n",""))
#         if (values['baseline_recs'] is None) or (values['steered_recs'] is None):
#             print(alpha, user)
    

In [16]:
outputs[0.1][279388]['baseline_recs']

KeyError: 0.1

In [77]:
book_dict['2342881']

{'title': 'The Secret Diaries of Charlotte Brontë',
 'title_without_series': 'The Secret Diaries of Charlotte Brontë',
 'isbn': '006164837X',
 'authors': [{'author_id': '806500', 'role': ''}],
 'language_code': 'eng'}

In [81]:
# Going to loop through books and see if the user's history is included in the recommendations
# result_df[result_df.user_id==46163]
# book_dict['2342881']

    

In [87]:
# result_df.head(2)
# user_title_dict

In [64]:
# data = json.loads((outputs[10][189064]['steered_recs']))
outputs[0.1][279388]['baseline_text'].split(str3)[1].replace("\n","")
# outputs[0.1][46163]['steered_text'].split(str3)[1].replace("\n","")

'{"Books": {    "title": "The Bone Collector",    "author": "Jeffery Deaver",    "genre": "Thriller"},{"Books": {    "title": "Fool Me Once",    "author": "Harlan Coben",    "genre": "Thriller"},{"Books": {    "title": "Star Trek: The Next Generation Technical Manual",    "author": "Michael Okuda",    "genre": "Reference"}}'

In [96]:
outputs[1][224839].keys()
# ['titles']

dict_keys(['baseline_text', 'steered_text', 'pre_hidden', 'post_hidden', 'baseline_recs', 'steered_recs', 'titles'])

In [113]:
# outputs[1].keys()
for alpha, user_ids in outputs.items():
    for user, values in user_ids.items():
        values['titles'] = user_title_dict[user]
        values['baseline_count'] = 0
        values['steered_count'] = 0
        for title in values['titles']:
            if title in values['baseline_text']:
                values['baseline_count'] += 1
            if title in values['steered_text']:
                values['steered_count'] += 1

In [101]:
default_value = {}
small_dict = {key: {'baseline':[], 'steered':[]} for key in outputs}
 # = {al_dict.keys()}
for k,v in outputs.items(): #alpha values

    for key, value in v.items(): # user IDs
        for key1, value1 in value.items(): #values per user ID
            if key1 == 'baseline_count':
                small_dict[k]['baseline'].append(value1)
            elif key1 == 'steered_count':
                small_dict[k]['steered'].append(value1)

In [116]:
for k,v in outputs.items(): #alpha values
    for key, value in v.items(): # user IDs
        if (value['baseline_count'] + value['steered_count']) > 6:
            print(k,key,value['baseline_count'],value['steered_count'])

0.1 163356 4 3
5 163356 4 3
10 163356 4 4


In [119]:
outputs[0.1][163356]['titles']

['Traveling Mercies: Some Thoughts on Faith',
 'Turtles All the Way Down',
 'Freckles (Limberlost #1)',
 'The Biggest Bear',
 'Tuesdays with Morrie',
 'The Book Thief',
 'The Trees of the Dancing Goats',
 'Darfur and the Crime of Genocide',
 'I Never Promised You a Rose Garden',
 'My Family and Other Animals (Corfu Trilogy, #1)']

In [123]:
outputs[10][163356]['baseline_text']
# outputs[0.1][163356]['titles']

'system\n\nCutting Knowledge Date: December 2023\nToday Date: 11 Nov 2025\n\nuser\n\nHi, I\'ve read and enjoyed the following books: Turtles All the Way Down by John Green,  \n    Darfur and the Crime of Genocide by John Hagan  and Freckles (Limberlost #1) by Gene Stratton-Porter.  Only return the 3 books you recommend in JSON format like {"Books": {\'title\':..., \'author\':...}}, and nothing else.assistant\n\n{\n  "Books": [\n    {"title": "The Perks of Being a Wallflower", "author": "Stephen Chbosky"},\n    {"title": "The Book Thief", "author": " Markus Zusak"},\n    {"title": "Island of the Blue Dolphins", "author": "Scott O\'Dell"}\n  ]\n}'

## Below is with the old outputs format 

In [23]:
# data_dict

In [27]:
# output
# search in user data for these users
for i in output:
    for k,v  in data_dict.items():
        counter = 0
        for j in v:
            if j in i['orig']:
                counter += 1
        if counter >= 3:
            i['hist'] = v
            i['hist_len'] = len(v)
            break
    


In [41]:
def extract_books(entry):
    # Case 1: structured JSON-like string
    if isinstance(entry, dict) and "Books" in entry:
        return entry["Books"]
    
    if isinstance(entry, str) and entry.strip().startswith("{"):
        try:
            data = json.loads(entry)
            if "Books" in data:
                return data["Books"]
        except json.JSONDecodeError:
            pass  # fall through to regex

    # Case 2: free-form text like "Title (Series, #N) by Author"
    pattern = r"([A-Z][^()]*?(?:\([^)]*\))?\s+by\s+[A-Z][^,\.]+)"
    matches = re.findall(pattern, entry)

    books = []
    for m in matches:
        # Split into title and author
        title_part, author = m.split(" by ", 1)
        books.append({"title": title_part.strip(), "author": author.strip()})
    
    return books

In [50]:
pattern = r"([A-Z][^()]*?(?:\([^)]*\))?\s+by\s+[A-Z][^,\.]+)"
for i in output:
    i['baseline_recs'] = [b.strip(" ,.\n") for b in re.findall(pattern, i['baseline_output'].split(spl_str)[1])]
    i['steered_recs'] = [b.strip(" ,.\n") for b in re.findall(pattern, i['steered_output'].split(spl_str)[1])]